In [1]:
import re
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [12]:
base_url = f'https://www.postnl.nl/over-postnl/werkenbij/vacatures-in-de-postbezorging-sortering/?searchterm=&zipcode=&distance=10'

def build_url(page):
    return base_url + f'&page={page}'


In [13]:
def get_page(page):
    try:
        url = build_url(page)
        r = requests.get(url)
#         print('Status code:', r.status_code)

        if r.status_code == 200:
#             print(f'Site found:\n{url}')
            return r
        else:
            print(f'Error opening:\n{url}')  

    except:
        print(f'Cannot open:\n{url}')
        

In [14]:
r = get_page(1)

In [15]:
r.text

'\n<!DOCTYPE html>\n<html lang="nl" xmlns:og="http://ogp.me/ns#">\n<head>\n<title>Vacatures in bezorging en sortering | PostNL</title>\n<meta name="google-site-verification" content="GWGxBXl92bzo7nltDfPaLKbugMtgYNzQ8Rbkuhm6iYk" />\n<meta name="websiteKey" content="postnl.nl" />\n<meta property="og:locale" content="nl_NL" />\n<meta property="og:url" content="https://www.postnl.nl/over-postnl/werkenbij/vacatures-in-de-postbezorging-sortering/?searchterm=&amp;zipcode=&amp;distance=10&amp;page=1" />\n<meta property="og:site_name" content="PostNL" />\n<meta name="twitter:image" content="https://www.postnl.nl/img/logo.png" />\n<meta property="og:image" content="https://www.postnl.nl/img/logo.png" />\n<meta property="og:title" content="Vacatures bezorgen en sorteren" />\n<meta name="twitter:title" content="Vacatures bezorgen en sorteren" />\n<meta name="twitter:description" content="Missing Resource with key: vacancyOverview-production-metaDescription" />\n<meta property="og:description" cont

In [16]:
soup = BeautifulSoup(r.text)

In [17]:
blocks = soup.find_all(id=re.compile('vacancy-\d+'))

In [18]:
len(blocks)

10

In [19]:
blocks[0]['id'].split('-')[1]

'20016'

In [20]:
def get_jobs(r):
    soup = BeautifulSoup(r.text)
    
    blocks = soup.find_all(id=re.compile('vacancy-\d+'))

    jobs = []
    for block in blocks:
        d = {}
        d['nr'] = block['id'].split('-')[1]
        d['title'] = block.find(class_='title').get_text()
        d['intro'] = block.find(class_='intro').get_text()
        d['plaats'] = block.find(itemprop='addressLocality').get_text()
        d['postcode'] = block.find(itemprop='postalCode').get_text()

        jobs.append(d)

    return jobs

In [21]:
get_jobs(r)

[{'nr': '20016',
  'title': 'Postbezorger Op Scooter',
  'intro': 'Postbezorger op de scooter is de ideale baan als je houdt van vrijheid en snelheid! Lees meer over deze parttime vacature en solliciteer direct.',
  'plaats': 'Andelst',
  'postcode': '6673 BW'},
 {'nr': '22239',
  'title': 'Postbezorger Op Scooter',
  'intro': 'Postbezorger op de scooter is de ideale baan als je houdt van vrijheid en snelheid! Lees meer over deze parttime vacature en solliciteer direct.',
  'plaats': 'Wolphaartsdijk',
  'postcode': '4471 CG'},
 {'nr': '22479',
  'title': 'Postbezorger',
  'intro': 'Wil je iets bijverdienen en ben je graag actief bezig? Word parttime postbezorger bij PostNL. Lees meer over deze vacature en solliciteer direct.',
  'plaats': 'Wemeldinge',
  'postcode': '4424 BB'},
 {'nr': '25096',
  'title': 'Postbezorger',
  'intro': 'Wil je iets bijverdienen en ben je graag actief bezig? Word parttime postbezorger bij PostNL. Lees meer over deze vacature en solliciteer direct.',
  'plaa

In [22]:
jobs = []
    
try:
     
    for page in range(1, 50):
        print(f'Scraping page {page}')
        r = get_page(page)

        jobs_on_page = get_jobs(r)
#         print(f'{len(jobs_on_page)} jobs found.')

        if len(jobs_on_page) == 0:
            break
        
        jobs.extend( jobs_on_page )
                
except:
    print('Done by exception')
    
finally:
    print('Done')
    

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Done


In [23]:
len(jobs)

304

In [24]:
df = pd.DataFrame(jobs)
df.set_index('nr', inplace=True)
df

,title,intro,plaats,postcode
nr,,,,
20016,Postbezorger Op Scooter,Postbezorger op de scooter is de ideale baan a...,Andelst,6673 BW
22239,Postbezorger Op Scooter,Postbezorger op de scooter is de ideale baan a...,Wolphaartsdijk,4471 CG
22479,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,Wemeldinge,4424 BB
25096,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,Valburg,6675 AW
25145,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,Sas van Gent,4551EH
...,...,...,...,...
29957,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,Genemuiden,8281 AT
29702,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,'s-Gravenhage,2544 EX
29963,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,'s-Gravenhage,2512 VX


In [27]:
df[df['plaats']=='Maastricht']

,title,intro,plaats,postcode
nr,,,,
29843,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,Maastricht,6229 PM
28397,Postbezorger,Wil je iets bijverdienen en ben je graag actie...,Maastricht,6212BL


In [28]:
df.groupby('plaats').count().sort_values('title', ascending=False).head(10)

,title,intro,postcode
plaats,,,
Nieuwegein,8,8,8
Zwolle,7,7,7
Haarlem,5,5,5
Breda,5,5,5
Oosterhout,4,4,4
Baarn,3,3,3
Wijchen,3,3,3
Utrecht,3,3,3
Tilburg,3,3,3


In [193]:
pd.DataFrame(df.melt())

,variable,value
0,title,Postbezorger Op Scooter
1,title,Postbezorger Op Scooter
2,title,Postbezorger
3,title,Postbezorger
4,title,Postbezorger
...,...,...
1415,postcode,2315 VC
1416,postcode,2341 VX
1417,postcode,2341 VX
1418,postcode,6815 GD


# Using Scrapy

In [92]:
import scrapy


class WerkenBijPostNlSpider(scrapy.Spider):

    name = 'jobs'
    
    start_urls = [
        build_url(1),
    ]

    def parse(self, response):
        for block in response.css('.block'):
            yield {
                'title': block.css('.title::text').get(),
                'intro': block.css('.class::text').get(),
                'plaats': block.css('*[itemprop="addressLocality"]::text').get(),
                'postcode': block.css('*[itemprop="postalCode"]::text').get(),
            }



In [93]:
scraper = WerkenBijPostNlSpider()

In [ ]:
scraper.